In [ ]:
#!/usr/bin/env python

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from teamName import getMyPosition as getPosition

nInst = 0
nt = 0
commRate = 0.0010
dlrPosLimit = 10000

def loadPrices(fn):
    global nt, nInst
    df = pd.read_csv(fn, sep='\s+', header=None, index_col=None)
    (nt, nInst) = df.shape
    return (df.values).T


pricesFile = "./prices.txt"
prcAll = loadPrices(pricesFile)
print("Loaded %d instruments for %d days" % (nInst, nt))

def calcPL(prcHist):
    cash = 0
    curPos = np.zeros(nInst)
    totDVolume = 0
    totDVolumeSignal = 0
    totDVolumeRandom = 0
    value = 0
    todayPLL = []
    (_, nt) = prcHist.shape
    for t in range(250, 501):
        prcHistSoFar = prcHist[:, :t]
        newPosOrig = getPosition(prcHistSoFar)
        curPrices = prcHistSoFar[:, -1]
        posLimits = np.array([int(x) for x in dlrPosLimit / curPrices])
        newPos = np.clip(newPosOrig, -posLimits, posLimits)
        deltaPos = newPos - curPos
        dvolumes = curPrices * np.abs(deltaPos)
        dvolume = np.sum(dvolumes)
        totDVolume += dvolume
        comm = dvolume * commRate
        cash -= curPrices.dot(deltaPos) + comm
        curPos = np.array(newPos)
        posValue = curPos.dot(curPrices)
        todayPL = cash + posValue - value
        todayPLL.append(todayPL)
        value = cash + posValue
        ret = 0.0
        if (totDVolume > 0):
            ret = value / totDVolume
        # print("Day %d value: %.2lf todayPL: $%.2lf $-traded: %.0lf return: %.5lf" %
        #      (t, value, todayPL, totDVolume, ret))
    pll = np.array(todayPLL)
    (plmu, plstd) = (np.mean(pll), np.std(pll))
    annSharpe = 0.0
    if (plstd > 0):
        annSharpe = np.sqrt(250) * plmu / plstd
    return (plmu, ret, plstd, annSharpe, totDVolume)


(meanpl, ret, plstd, sharpe, dvol) = calcPL(prcAll)
score = meanpl - 0.1*plstd
print("=====")
print("mean(PL): %.1lf" % meanpl)
print("return: %.5lf" % ret)
print("StdDev(PL): %.2lf" % plstd)
print("annSharpe(PL): %.2lf " % sharpe)
print("totDvolume: %.0lf " % dvol)
print("Score: %.2lf" % score)

In [ ]:
from itertools import cycle 
window_size = 250
instrument_pairs = [27, 38]
adjustment = 50

# Calculate percentage returns
returns = np.diff(prcAll, axis=1) / prcAll[:, :-1] * 100

# Calculate rolling window statistics
rolling_window1 = returns[instrument_pairs[0], -window_size:]
rolling_window2 = returns[instrument_pairs[1], -window_size:]

moving_average1 = np.mean(rolling_window1)
moving_average2 = np.mean(rolling_window2)

spread = returns[instrument_pairs[0], -1] - returns[instrument_pairs[1], -1]
spread_moving_average = moving_average1 - moving_average2
spread_std_dev = np.std(rolling_window1 - rolling_window2)

# Plotting
instruments_to_plot = [27, 38]
colors = cycle(['b', 'k', 'r', 'c', 'm', 'y', 'k'])
plt.figure(figsize=(20, 12))

daily_returns = np.diff(prcAll, axis=1) / prcAll[:, :-1] * 100

""" df = pd.DataFrame(daily_returns.T)
window_size_plot = 100
moving_averages = df.rolling(window=window_size_plot).mean()
moving_std_devs = df.rolling(window=window_size_plot).std()
 """
for i in instruments_to_plot:
    color = next(colors)
    mean_price = np.mean(daily_returns[i, :])
    std_dev = np.std(daily_returns[i, :])
    #plt.plot(daily_returns[i, :], color=color, label=f'Instrument {i} Returns')
    #plt.plot(moving_averages[i], color=color, label=f'Instrument {i} Moving Average')
   # plt.plot(moving_averages[i] + 1 * moving_std_devs[i], color=color, linestyle=':', label=f'Instrument {i} +1 Std Dev')
    #plt.plot(moving_averages[i] - 1 * moving_std_devs[i], color=color, linestyle=':', label=f'Instrument {i} -1 Std Dev')

# Adding spread plot
spread_values = returns[instrument_pairs[0]] - returns[instrument_pairs[1]]
moving_average_spread = np.mean(spread_values[:])
std_dev_spread = np.std(spread_values[:])

plt.plot(spread_values, color='cyan', label='Spread', linewidth=2)
plt.axhline(y=moving_average_spread, color='red', linestyle='-', linewidth=2, label='Spread Moving Average')
plt.axhline(y=moving_average_spread + 2.5 * std_dev_spread, color='red', linestyle='--', label='Spread +2 Std Dev')
plt.axhline(y=moving_average_spread - 2.5 * std_dev_spread, color='red', linestyle='--', label='Spread -2 Std Dev')

plt.xlabel('Time')
plt.ylabel('Returns (%)')
plt.title('Pairs Trading Strategy: Returns, Moving Averages, and Spread')
plt.legend()
plt.show()

In [ ]:
instruments_to_plot = [0]

daily_returns = np.diff(prcAll) / prcAll[:, :-1]

plt.figure(figsize=(20,12))
for i in instruments_to_plot:
    plt.plot(100 * daily_returns[i, :])

plt.xlabel('Days')
plt.ylabel('% Return')
plt.title('Daily Returns of Each Instrument from Day 1 to 500')
plt.tight_layout()
plt.show()